# Practical Exercise (Optional): Create Modern Modular Network 

In this exercise, you are going to make use of some architecural elements that you learnt, such as residual connection, and separable convolution, and combine them into a building block, and use this building block to construct a modular network. 

This building block is similar to what [Xception](https://arxiv.org/abs/1610.02357) is using. 

Here is the simplified view of our network: 

![xception](https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/iti107/resources/mini_xception.png)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

**Exercise 1**

Let us the define our xception block first. Here are the steps you will need to do: 
1. First save the output from previous block (or if this is the first block, then output from previous convolutional/maxpooling layer) and use this as skip connection to be added later
2. Add the 2 separable convolutional and the maxpool layer. 
3. Add BatchNormalization after each separable convolutional layer (as practiced by modern architecture).  You can add BatchNormalization before or after Activation layer.
4. Use 1x1 conv to change the depth of the skip connection to match that of the rest of layers and add the skip connection to the output from MaxPooling layer. You will also need to reduce the size to match the output from MaxPooling layer, by using a stride size of 2
5. As recommended by author of ResNet, the skip connection should be added before the Activation function. 

In [ ]:
def xception_block(x, depth): 
    """
    Parameters:
    -----------
    x: output from the previous block or previous layer, considered as input to this Xception block
       it is a tensor of shape (batch, h, w, channels)
    depth: number of channels, it is an int value
    """
    
    # save input to be used as skip connection
    skip_connection = ?
    
    # add the first separable convolutional 2D layer
    x = ?
    
    # add the second separable convolutional 2D layer, without the activation layer
    x = ?
    
    # add the maxpooling 2d layer, with stride of 2
    x = ?
    
    # adjust the size and depth using 1x1 convolution to match the output from last maxpooling layer
    residual = ?
    
    # add the skip connection 
    x = layers.add([x, residual])  # Add back residual
    
    # add the activation layer
    x = ?
    
    return x # Set aside next residual

**Exercise 2:**

We will then build our complete network using the xception block defined in the previous exercise: 
1. Define the input layer with appropriate shape
2. Define a rescaling layer to normalize the input to (0,1)
3. Define the entry blocks that consist of 2 convolutional 2D layers 
4. Add the series of xception blocks, with different depths `[128,256,512,728]`
5. Add the last Separable convolutional 2D layer 
6. Add GlobalAveragePooling2D layer before connected to output Dense Layer
7. Use Dropout for the dense layer 



In [ ]:
def make_model(input_shape, num_classes): 
    inputs = keras.Input(shape=input_shape)
    
    # Entry block
    x = ?
    
    # build a series of xception blocks with different sizes
    for size in [128, 256, 512, 728]:
        x = ?? 
    
    outputs = ?
    
    return keras.Model(inputs, outputs)

In [ ]:
image_size = (128,128)

model = make_model(input_shape= image_size + (3,), num_classes=2)

keras.utils.plot_model(model, show_shapes=True)

### Create train and validation dataset

We will go ahead and download the same dataset, and setup the training and validation dataset.

In [ ]:
import os 

dataset_URL = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/cats_and_dogs_subset.tar.gz'
tf.keras.utils.get_file(origin=dataset_URL, extract=True, cache_dir='.')
dataset_folder = os.path.join('datasets', 'cats_and_dogs_subset')

In [ ]:
batch_size = 8

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_folder,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_folder,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary'
)

Ok, everything is ready. Now we are ready to put our mini-xception network to test and see if perform better than our previous 'traditional' CNN architecture. 

In [ ]:
def create_tb_callback(): 

    import os
    
    root_logdir = os.path.join(os.curdir, "tb_logs")

    def get_run_logdir():    # use a new directory for each run
        
        import time
        
        run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
        return os.path.join(root_logdir, run_id)

    run_logdir = get_run_logdir()

    tb_callback = tf.keras.callbacks.TensorBoard(run_logdir)

    return tb_callback

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="bestcheckpoint",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


# compile our model with loss and optimizer 
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)


model.fit(
    train_ds, epochs=30, 
    validation_data=val_ds,
    callbacks=[model_checkpoint_callback]
)

In [ ]:
best_checkpoint = 'bestcheckpoint'

model.load_weights(best_checkpoint)
model.evaluate(val_ds)